# Instruction Fine-tuning sobre un LLM Base

<div style="background-color:#D9EEFF;color:black;padding:2%;">
<h2>Fine Tuning Medical database</h2>

En este caso práctico, se proporne la realización de instruction fine-tuning sobre el LLM [Flan-T5-small](https://huggingface.co/google/flan-t5-small) con el objetivo de que sea capaz de iferir alguna enfermedad o afeccion dado unos sintomas.

</div>

## 0. Instalación de librerías externas

In [2]:
#!pip install transformers
#!pip install sentencepiece
#!pip install accelerate
#!pip install datasets
#!pip install evaluate
#!pip install rouge_score

In [5]:
#!pip install --upgrade jupyter ipywidgets

## 1. Comportamiento de [Flan-T5-small](https://huggingface.co/google/flan-t5-small) sin Fine-tuning

### Lectura del modelo y tokenizador

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Importamos el tokenizador
tokenizer_FT5 = T5Tokenizer.from_pretrained("google/flan-t5-small")

# Importamos el modelo pre-entrenado
model_FT5 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")

# Lectura Dataset

In [8]:
import pandas as pd
import numpy as np

data = pd.read_csv('./web_scrapping/cls_llm_dataset_final4.csv')

In [10]:
print(data.shape)
data.head()

(1158, 2)


,Text,Label
0,Manchado o sangrado vaginal\r\nDolor o calambr...,Aborto espontáneo
1,"Dolor de muela intenso, persistente y grave qu...",Absceso dental
2,"Robar, falsificar o vender recetas\r\nTomar do...",Abuso de drogas recetadas
3,"Incapacidad para tragar (disfagia), que puede ...",Acalasia
4,Los cambios en la piel son los únicos signos d...,Acantosis pigmentaria


In [11]:
n_register = np.random.randint(len(data))
text = data['Text'][n_register]
label = data['Label'][n_register]


print(f'\tNumero de registro: {n_register}')
print(f'*'*40)
print(f'\tTexto:\n{text}\n\n')
print(f'\t-> Label: {label}')


	Numero de registro: 138
****************************************
	Texto:
Aumento de peso
Debilidad muscular
Estrías rosadas o púrpuras en la piel
Cambios hormonales en las mujeres que podrían causar exceso de vello facial, pérdida de cabello en la cabeza y períodos menstruales irregulares
Cambios hormonales en los hombres que podrían causar agrandamiento del tejido mamario y encogimiento de los testículos
Náuseas
Vómitos
Hinchazón abdominal
Dolor de espalda
Fiebre
Pérdida de apetito
Pérdida de peso sin intentarlo


	-> Label: Cáncer de la glándula suprarrenal


### Generación de texto

In [19]:
#!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu125

In [20]:
import torch
print(torch.version.cuda)

None


In [12]:
context = data['Text'][0]

In [24]:
context = "Tengo sangrado vaginal en el primer trimestre. ¿Qué enfermedad podría tener?"

In [14]:
context = data['Text'][2]

In [15]:
question = '¿Qué enfermedad o afección podría tener?'

In [25]:
prompt_template = f"Responde basado en contexto:\n\n{context}\n\n{question}"

# Verificar si CUDA está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"

# Tokenizamos el prompt
prompt_tokens = tokenizer_FT5(prompt_template, return_tensors="pt").input_ids.to(device)

# Generamos los siguientes tokens
outputs = model_FT5.generate(prompt_tokens, max_length=200)

# Transformamos los tokens generados en texto
print(tokenizer_FT5.decode(outputs[0]))

<pad> enfermedad o afección</s>


## 2. Selección y preparación del conjunto de datos

### Formato del conjunto de datos

Es habitual utilizar plantillas que proponen los desarrolladores de los LLM para diseñar nuestros ejemplos de entrenamiento: https://github.com/google-research/FLAN/blob/main/flan/v2/flan_templates_branched.py

In [26]:
# prompt_template = f"Responde basado en contexto:\n\n{context}\n\n{question}"
def convert_to_template(context, question):
    return f"Responde basado en contexto:\n\n{context}\n\n{question}"

In [27]:
data['prompt'] = data.apply(lambda x: convert_to_template(x['Text'], question), axis=1)

In [28]:
print(data['prompt'][0])

Responde basado en contexto:

Manchado o sangrado vaginal
Dolor o calambres en el abdomen o la parte inferior de la espalda
Fluidos o tejidos que salen por la vagina
Si ha salido tejido fetal por la vagina, colócalo en un recipiente limpio y llévalo al consultorio de tu profesional de salud o al hospital para que se lo analice.
Recuerda que la mayoría de las mujeres que presentan manchado o sangrado vaginal durante el primer trimestre continúan teniendo embarazos exitosos.

¿Qué enfermedad o afección podría tener?


In [29]:
data.head()

,Text,Label,prompt
0,Manchado o sangrado vaginal\r\nDolor o calambr...,Aborto espontáneo,Responde basado en contexto:\n\nManchado o san...
1,"Dolor de muela intenso, persistente y grave qu...",Absceso dental,Responde basado en contexto:\n\nDolor de muela...
2,"Robar, falsificar o vender recetas\r\nTomar do...",Abuso de drogas recetadas,"Responde basado en contexto:\n\nRobar, falsifi..."
3,"Incapacidad para tragar (disfagia), que puede ...",Acalasia,Responde basado en contexto:\n\nIncapacidad pa...
4,Los cambios en la piel son los únicos signos d...,Acantosis pigmentaria,Responde basado en contexto:\n\nLos cambios en...


In [30]:
data_cls = data[['prompt', 'Label']]
data_cls.head()

,prompt,Label
0,Responde basado en contexto:\n\nManchado o san...,Aborto espontáneo
1,Responde basado en contexto:\n\nDolor de muela...,Absceso dental
2,"Responde basado en contexto:\n\nRobar, falsifi...",Abuso de drogas recetadas
3,Responde basado en contexto:\n\nIncapacidad pa...,Acalasia
4,Responde basado en contexto:\n\nLos cambios en...,Acantosis pigmentaria


In [32]:
!pip install scikit-learn

^C


In [31]:
# Dividimos el conjunto de datos en entrenamiento,test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data_cls, test_size=0.2, random_state=42)

ModuleNotFoundError: No module named 'sklearn'

In [29]:
# Convertimos a formato DatasetDict
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

ds = DatasetDict()

ds['train'] = train_dataset
ds['test'] = test_dataset

In [30]:
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 926
    })
    test: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 232
    })
})

In [31]:
ds['train']['prompt'][1]

'Responde basado en contexto:\n\nProtuberancia en los puntos blandos (fontanelas) del cráneo del bebé\r\nNáuseas y vómitos\r\nRigidez corporal\r\nAlimentación deficiente o no despertarse para alimentarse\r\nIrritabilidad\r\nCuándo debes consultar a un médico\r\nObtén atención médica inmediata si tienes alguno de los síntomas más graves asociados con la encefalitis. Ante síntomas como dolor de cabeza intenso, fiebre y alteración del estado de conciencia, se requiere atención urgente.\r\nLos bebés y niños pequeños que presenten algún signo o síntoma de encefalitis deben recibir atención médica de urgencia.\n\n¿Qué enfermedad o afección podría tener?'

In [32]:
ds['train']['Label'][1]

'Encefalitis'

In [33]:
# Reducimos el conjunto de datos
NUM_EJ_TRAIN = 926
NUM_EJ_VAL = 232


# Subconjunto de entrenamiento
ds['train'] = ds['train'].select(range(NUM_EJ_TRAIN))

# Subconjunto de validación
ds['test'] = ds['test'].select(range(NUM_EJ_VAL))

In [34]:
ds['test']['prompt'][1]

'Responde basado en contexto:\n\nHemorragia vaginal, aunque puede ser que no haya ninguna\r\nDolor abdominal\r\nDolor de espalda\r\nSensibilidad o rigidez uterina\r\nContracciones uterinas, a menudo una tras otra\r\nDolor abdominal y dolor de espalda que a menudo comienzan repentinamente. La cantidad de sangrado vaginal puede variar mucho y no necesariamente indica la cantidad de placenta que se ha separado del útero. Es posible que la sangre quede atrapada dentro del útero así que, incluso con un desprendimiento placentario grave, podría no haber sangrado visible.\r\nEn algunos casos, el desprendimiento placentario se desarrolla lentamente (desprendimiento crónico), lo que puede causar un ligero e intermitente sangrado vaginal. Es posible que el bebé no crezca tan rápido como se esperaba y que tenga un bajo nivel de líquido amniótico u otras complicaciones.\r\nCuándo consultar al médico\r\nBusca atención de emergencia si tienes signos o síntomas de desprendimiento de placenta.\n\n¿Qué

In [35]:
ds['test']['Label'][1]

'Desprendimiento de placenta'

### 2.3. Tokenización del conjunto de datos

In [36]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

In [37]:
from datasets import concatenate_datasets

# Calculamos el tamaño máximo de prompt
prompts_tokens = concatenate_datasets([ds["train"], ds["test"]]).map(lambda x: tokenizer(x["prompt"], truncation=True), batched=True)
max_token_len = max([len(x) for x in prompts_tokens["input_ids"]])
print(f"Maximo tamaño de prompt: {max_token_len}")

# Calculamos el tamaño máximo de completion
completions_tokens = concatenate_datasets([ds["train"], ds["test"]]).map(lambda x: tokenizer(x["Label"], truncation=True), batched=True)
max_completion_len = max([len(x) for x in completions_tokens["input_ids"]])
print(f"Maximo tamaño de completion: {max_completion_len}")

Map:   0%|          | 0/1158 [00:00<?, ? examples/s]

Maximo tamaño de prompt: 512


Map:   0%|          | 0/1158 [00:00<?, ? examples/s]

Maximo tamaño de completion: 38


In [38]:
def padding_tokenizer(datos):
  # Tokenizar inputs (prompts)
  model_inputs = tokenizer(datos['prompt'], max_length=max_token_len, padding="max_length", truncation=True)

  # Tokenizar labels (completions)
  model_labels = tokenizer(datos['Label'], max_length=max_completion_len, padding="max_length", truncation=True)

  # Sustituimos el caracter de padding de las completion por -100 para que no se tenga en cuenta en el entrenamiento
  model_labels["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in model_labels["input_ids"]]

  model_inputs['labels'] = model_labels["input_ids"]

  return model_inputs

In [39]:
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 926
    })
    test: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 232
    })
})

In [40]:
ds_tokens = ds.map(padding_tokenizer, batched=True , remove_columns=['prompt','Label'])
#remove_columns=['text', 'summary', 'topic', 'url', 'title', 'date', 'prompt']

Map:   0%|          | 0/926 [00:00<?, ? examples/s]

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

In [41]:
ds_tokens

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 926
    })
    test: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 232
    })
})

## 3. Fine-tuning del modelo

### 3.1. Lectura del modelo

In [42]:
from transformers import AutoModelForSeq2SeqLM

# Cargamos el modelo
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

### 3.2. Evaluación durante el entrenamiento

A continuación implementamos un conjunto de funciones auxiliares para evluar los resultados durante el proceso de entrenamiento

In [43]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('punkt_tab')

# Metrica de evaluación
metric = evaluate.load("rouge")

# Funciona auxiliar para preprocesar el texto
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum espera una nueva línea después de cada frase
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Reemplazamos -100 en las etiquetas porque no podemos decodificarlo
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Preprocesamos el texto
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


### 3.3. Lectura y adaptación de los datos para el entrenamiento

In [45]:
from transformers import DataCollatorForSeq2Seq

# Ignoramos los tokens relacionados con el padding durante el proceso de entrenamiento para los prompts
label_pad_token_id = -100

# Recolector de datos para el entrenamiento del modelo
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


### Preparación y ejecución del fine-tuning (entrenamiento)

In [52]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["WANDB_DISABLED"] = "true"

REPOSITORY="/content/drive/MyDrive/CURSOS UDEMY/IA GENERATIVA - UDEMY/Ciclo de Vida de un Proyecto de IA/MedicalT5_Intruction_Fine_Tuning_LLM"

# Definimos las opciones del entrenamiento
training_args = Seq2SeqTrainingArguments(
    # Hiperprámetros del entrenamiento
    output_dir=REPOSITORY,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,  # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=20,
    # Estrategias de logging y evaluación
    logging_dir=f"{REPOSITORY}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Creamos la instancia de entrenamiento
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=ds_tokens["train"],
    eval_dataset=ds_tokens["test"],
    compute_metrics=compute_metrics,
    processing_class=tokenizer,  # <-- Aquí se usa processing_class en lugar de tokenizer
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [53]:
# Guardamos el tokenizador en disco para utilizarlo posteriormente
tokenizer.save_pretrained(f"{REPOSITORY}/Medical_tokenizer")

('/content/drive/MyDrive/CURSOS UDEMY/IA GENERATIVA - UDEMY/Ciclo de Vida de un Proyecto de IA/MedicalT5_Intruction_Fine_Tuning_LLM/Medical_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/CURSOS UDEMY/IA GENERATIVA - UDEMY/Ciclo de Vida de un Proyecto de IA/MedicalT5_Intruction_Fine_Tuning_LLM/Medical_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/CURSOS UDEMY/IA GENERATIVA - UDEMY/Ciclo de Vida de un Proyecto de IA/MedicalT5_Intruction_Fine_Tuning_LLM/Medical_tokenizer/spiece.model',
 '/content/drive/MyDrive/CURSOS UDEMY/IA GENERATIVA - UDEMY/Ciclo de Vida de un Proyecto de IA/MedicalT5_Intruction_Fine_Tuning_LLM/Medical_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/CURSOS UDEMY/IA GENERATIVA - UDEMY/Ciclo de Vida de un Proyecto de IA/MedicalT5_Intruction_Fine_Tuning_LLM/Medical_tokenizer/tokenizer.json')

In [54]:

# Iniciamos el entrenamiento
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.894426,36.475400,24.103400,36.510300,36.488600,9.616379
2,No log,1.790589,40.145100,25.663100,40.111200,40.005000,10.103448
3,No log,1.749821,43.430900,27.308000,43.373500,43.220700,10.120690
4,No log,1.715682,43.782600,26.960700,43.968500,43.765700,10.379310
5,1.979600,1.698976,44.483500,27.542000,44.504000,44.398700,10.400862
6,1.979600,1.685845,44.696400,27.681300,44.762100,44.623600,10.443966
7,1.979600,1.672030,46.100600,28.604000,46.235300,46.104700,10.301724
8,1.979600,1.676674,45.203100,27.807200,45.306300,45.250100,10.293103
9,1.652000,1.664924,46.428700,29.226800,46.523100,46.496400,10.560345
10,1.652000,1.659609,46.194600,29.726500,46.292300,46.254900,10.564655


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=2320, training_loss=1.588006775954674, metrics={'train_runtime': 1195.4372, 'train_samples_per_second': 15.492, 'train_steps_per_second': 1.941, 'total_flos': 3442692147118080.0, 'train_loss': 1.588006775954674, 'epoch': 20.0})

## 4. Generación de texto con Flan-T5 Fine-tuned y evaluación

### Lectura del modelo y del tokenizador

In [58]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

REPOSITORY="/content/drive/MyDrive/CURSOS UDEMY/IA GENERATIVA - UDEMY/Ciclo de Vida de un Proyecto de IA/MedicalT5_Intruction_Fine_Tuning_LLM"

# Importamos el tokenizador
tokenizer_FT5_FT = T5Tokenizer.from_pretrained(f"{REPOSITORY}/Medical_tokenizer")

# Importamos el modelo con fine-tuning
model_FT5_FT = T5ForConditionalGeneration.from_pretrained(f"{REPOSITORY}/checkpoint-2320", device_map="auto")

### Generación de texto

In [65]:
context = """
    Sudoración \
    Escalofríos y temblores \
    Dolor de cabeza \
    Dolores musculares \
    Pérdida de apetito \
    Irritabilidad \
    Deshidratación \
    Debilidad general \
"""
# Fiebre

In [66]:
context = """
Manchado o sangrado vaginal
Dolor o calambres en el abdomen o la parte inferior de la espalda
Fluidos o tejidos que salen por la vagina
Si ha salido tejido fetal por la vagina, colócalo en un recipiente limpio y llévalo al consultorio de tu profesional de salud o al hospital para que se lo analice.
Recuerda que la mayoría de las mujeres que presentan manchado o sangrado vaginal durante el primer trimestre continúan teniendo embarazos exitosos.
"""
# Aborto

In [ ]:
context = """"
    Dolor de cabeza. \
    Tos persistente y seca. \
    Falta de aire. \
    Cansancio y debilidad. \
    Congestión o goteo nasal. \
    Dolor de garganta. \
    Dolor en los ojos. \
"""
# gripe(influencia)

In [67]:
data

,Text,Label,prompt
0,Manchado o sangrado vaginal\r\nDolor o calambr...,Aborto espontáneo,Responde basado en contexto:\n\nManchado o san...
1,"Dolor de muela intenso, persistente y grave qu...",Absceso dental,Responde basado en contexto:\n\nDolor de muela...
2,"Robar, falsificar o vender recetas\r\nTomar do...",Abuso de drogas recetadas,"Responde basado en contexto:\n\nRobar, falsifi..."
3,"Incapacidad para tragar (disfagia), que puede ...",Acalasia,Responde basado en contexto:\n\nIncapacidad pa...
4,Los cambios en la piel son los únicos signos d...,Acantosis pigmentaria,Responde basado en contexto:\n\nLos cambios en...
...,...,...,...
1153,No se encontraron síntomas,Sarcoma de Kaposi,Responde basado en contexto:\n\nNo se encontra...
1154,No se encontraron síntomas,Sarcoma epitelioide,Responde basado en contexto:\n\nNo se encontra...
1155,No se encontraron síntomas,Sarcoma sinovial,Responde basado en contexto:\n\nNo se encontra...
1156,No se encontraron síntomas,Válvula aórtica bicúspide,Responde basado en contexto:\n\nNo se encontra...


In [96]:
n_register = np.random.randint(len(data))
text = data['Text'][n_register]
label = data['Label'][n_register]


print(f'\tNumero de registro: {n_register}')
print(f'*'*40)
print(f'\tTexto:\n{text}\n\n')
print(f'\t-> Label: {label}')


	Numero de registro: 330
****************************************
	Texto:
Dificultad repentina para respirar
Exceso de líquido en los pulmones (edema pulmonar)
Presión arterial baja repentina
Falla repentina del corazón para bombear sangre de manera efectiva (colapso cardiovascular)
Problemas potencialmente mortales con la coagulación de la sangre (coagulopatía intravascular diseminada)
Sangrado del útero, incisión por cesárea o lugares de vías intravenosas (IV)
Estado mental alterado, como ansiedad o sensación de fatalidad
Escalofríos
Frecuencia cardíaca acelerada o alteraciones en el ritmo de la frecuencia cardíaca
Sufrimiento fetal, como una frecuencia cardíaca lenta u otras anomalías de la frecuencia cardíaca fetal
Convulsiones
Pérdida del conocimiento


	-> Label: Embolia amniótica


In [84]:
context = '''
Fiebre
Malestar general
Dolor de cabeza
Fatiga intensa
Dolor de espalda intenso
Vómitos, posiblemente
'''

In [85]:
question = '¿Qué enfermedad o afección podría tener?'

In [86]:
# Construimos el prompt conforme a la plantilla de fine-tuning
prompt_template = f"Responde basado en contexto:\n\n{context}\n\n{question}"

# Tokenizamos el prompt
prompt_tokens = tokenizer_FT5_FT(prompt_template, return_tensors="pt").input_ids.to("cuda")

# Generamos los siguientes tokens
outputs = model_FT5_FT.generate(prompt_tokens, max_length=300)

# Transformamos los tokens generados en texto
print(tokenizer_FT5_FT.decode(outputs[0]))

<pad> Cáncer de ni<unk> o</s>


In [73]:
# Creamos una funcion para predecir un contexto
def predict_context(context, question):
    prompt_template = f"Responde basado en contexto:\n\n{context}\n\n{question}"

  # Tokenizamos el prompt
    prompt_tokens = tokenizer_FT5_FT(prompt_template, return_tensors="pt").input_ids.to("cuda")

    # Generamos los siguientes tokens
    outputs = model_FT5_FT.generate(prompt_tokens, max_length=300)

    # Transformamos los tokens generados en texto
    return tokenizer_FT5_FT.decode(outputs[0])


Probando Algunas preguntas

In [95]:
context = '''
No lograr aliviar los síntomas de la rinitis alérgica (fiebre del heno).
Los medicamentos para la alergia no me alivian o me causan efectos secundarios molestos.
Tengo otro trastorno que puede empeorar la rinitis alérgica, como pólipos nasales, asma o infecciones frecuentes de los senos paranasales.
'''
# Fiebre del heno
question = '¿Qué enfermedad o afección podría tener?'
respuesta =  predict_context(context,question)
print(respuesta)

<pad> Erinitis alérgica</s>


In [93]:
context = '''
Dolor en la parte baja del abdomen
Dolor al orinar
Necesidad de orinar a menudo
Dificultad para orinar o interrupción del flujo de orina
Sangre en la orina
Orina turbia o de un color oscuro anormal
'''
# Cálculos en la vejiga
question = '¿Qué enfermedad o afección podría tener?'
respuesta =  predict_context(context,question)
print(respuesta)

<pad> Cáncer de o<unk> do</s>


In [97]:
context = '''
Dificultad repentina para respirar
Exceso de líquido en los pulmones (edema pulmonar)
Presión arterial baja repentina
Falla repentina del corazón para bombear sangre de manera efectiva (colapso cardiovascular)
Problemas potencialmente mortales con la coagulación de la sangre (coagulopatía intravascular diseminada)
Sangrado del útero, incisión por cesárea o lugares de vías intravenosas (IV)
Estado mental alterado, como ansiedad o sensación de fatalidad
Escalofríos
Frecuencia cardíaca acelerada o alteraciones en el ritmo de la frecuencia cardíaca
'''
# Embolia amniótica
question = '¿Qué enfermedad o afección podría tener?'
respuesta =  predict_context(context,question)
print(respuesta)

<pad> Trombosis en el n<unk> cleo</s>


In [91]:
context = '''
Amnesia disociativa. El síntoma principal es una pérdida de memoria que es más grave que un olvido normal y que no puede justificarse por la existencia de una enfermedad. No puedes recordar información sobre ti ni sobre acontecimientos y personas de tu vida, en especial los relacionados con un momento traumático. La amnesia disociativa puede ser específica de acontecimientos producidos en un cierto momento, como combates intensos, o, con menor frecuencia, puede tratarse de la pérdida completa de la memoria sobre ti mismo. A veces puede implicar que te traslades o deambules en un estado de confusión que te aleje de tu vida (fuga disociativa). El episodio de amnesia generalmente se presenta en forma repentina y puede durar minutos, horas o, rara vez, meses o años.
Amnesia disociativa.
Trastorno de identidad disociativo. Este trastorno, antes conocido como «trastorno de personalidad múltiple», se caracteriza por «alternar» diferentes identidades. Es posible que sientas la presencia de dos o más personas que hablan o viven en tu cabeza y que sientas que estas identidades te poseyeron. Cada identidad puede tener un nombre, una historia personal y características únicas, entre ellas, diferencias obvias de voz, género, tratos e incluso cualidades físicas, como la necesidad de usar lentes. También hay diferencias en cuanto a la familiaridad de cada identidad con las demás. Las personas con trastorno de identidad disociativo, en general, también tienen amnesia disociativa y, a menudo, sufren fuga disociativa.
Trastorno de identidad disociativo.
Trastorno de despersonalización-desrealización. Este trastorno implica una sensación continua o episódica de desconexión o de estar fuera de ti mismo, al observar tus acciones, sentimientos, pensamientos y a ti mismo desde cierta distancia como si estuvieras mirando una película (despersonalización). Es posible que otras personas y cosas que te rodean se perciban distantes, borrosas o como en un sueño, que el tiempo transcurra más lenta o más rápidamente y que el mundo parezca irreal (desrealización). Puedes sentir despersonalización, desrealización o ambas. Es posible que los síntomas, que pueden ser profundamente angustiantes, duren solo unos momentos o que vayan y vengan a lo largo de los años.
Trastorno de despersonalización-desrealización.
'''
question = '¿Qué enfermedad o afección podría tener?'
respuesta =  predict_context(context,question)
print(respuesta)

Token indices sequence length is longer than the specified maximum sequence length for this model (1009 > 512). Running this sequence through the model will result in indexing errors


<pad> Trastorno de identidad disociativo</s>


In [99]:
context = '''
Dolor en el pecho al respirar o toser
Desorientación o cambios de percepción mental (en adultos de 65 años o más)
Tos que puede producir flema
Fatiga
Fiebre, transpiración y escalofríos con temblor
Temperatura corporal más baja de lo normal (en adultos mayores de 65 años y personas con un sistema inmunitario débil)
Náuseas, vómitos o diarrea
Dificultad para respirar
'''
# Neumonia
question = '¿Qué enfermedad o afección podría tener?'
respuesta =  predict_context(context,question)
print(respuesta)

<pad> Fósiles en el intestino</s>


### Evaluación con el subconjunto de pruebas

In [89]:
import torch

# Cambiamos el modelo al modo de evaluación
model_FT5_FT.eval()

# Definir tamaño del lote
batch_size = 8

all_predictions = []

# Deshabilitamos el entrenamiento y obtenemos las completions
with torch.no_grad():
  for i in range(0, len(ds_tokens["test"]["input_ids"]), batch_size):
        # Extraemos el lote actual
        input_ids_batch = torch.tensor(ds_tokens["test"]["input_ids"][i:i+batch_size], device='cuda:0')

        # Obtenemos las predicciones del modelo
        outputs = model_FT5_FT.generate(input_ids_batch)

        # Concatenemos las predicciones
        all_predictions.extend(outputs)

# Calculamos las metricas
labels = np.array(ds_tokens['test']['labels'])

# Pad the predictions to the same length
max_len = max(len(pred) for pred in all_predictions)
padded_predictions = [np.pad(pred.cpu().numpy(), (0, max_len - len(pred)), 'constant', constant_values=tokenizer.pad_token_id) for pred in all_predictions]

completions = np.array(padded_predictions)

metrics = compute_metrics((completions, labels))

print(metrics)

{'rouge1': 45.8094, 'rouge2': 29.484, 'rougeL': 45.978, 'rougeLsum': 45.9549, 'gen_len': 10.28448275862069}
